### Import modules used in this notebook

In [78]:
# Import libraries
import os
import re
import requests
import gc
import io
import ast
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4
import warnings
warnings.filterwarnings("ignore")
import sys

In [79]:
# Import OOINet library
sys.path.append("c:\\Users\\cooleyky\\Documents\\GitHub\\OOINet") # this is what was missing from the steps I followed to install ooinet and ooi-data-explorations as local dev repo
from ooinet import M2M
from ooinet.Instrument.common import process_file

In [80]:
# Import functions from ooi-data-explorations library
sys.path.append("c:\\Users\\cooleyky\\Documents\\GitHub\\ooi-data-explorations\\python") # why did the initial install not include this?
from ooi_data_explorations.uncabled.process_dosta import dosta_datalogger
from ooi_data_explorations.combine_data import combine_datasets
from ooi_data_explorations import common as ooi_common

In [81]:
# Import dask tools and ProgressBar
import dask
from dask.diagnostics import ProgressBar

### Define data parameters and routines

In [82]:
# Setup parameters needed to request data
refdes = "CP01CNSM-MFD37-03-CTDBPD000"              # Coastal Pioneer Array (NES) - Central Surface Mooring CTD Bottom-pumped, is this the same as site, node, sensor?
method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "ctdbp_cdef_instrument_recovered"          # name of data stream
# Site, node, and sensor info from deconstructed reference designator
site = "CP01CNSM"
node = "MFD37"
sensor = "03-CTDBPD000"

In [83]:
# Generic preprocessing routine to do some generic dataset cleaning/processing
@dask.delayed
def preprocess(ds):
    ds = xr.open_dataset(ds)
    ds = process_file(ds)
    return ds

### QARTOD in Production: Request data from the THREDDS catalog

##### Using mostly OOINet module

In [84]:
# Use the gold copy THREDDs datasets
thredds_url = M2M.get_thredds_url(refdes, method, stream, goldCopy=True)

# Get the THREDDs catalog
thredds_catalog = M2M.get_thredds_catalog(thredds_url)

In [8]:
# Clean the THREDDs catalog
sensor_files, ancillary_files = M2M.clean_catalog(thredds_catalog, stream) 
# removes entries from thredds_catalog if they do not match the stream, or are used in processing data from the selected stream
sensor_files

['catalog.html?dataset=ooigoldcopy/public/CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0001_CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered_20131121T181601-20140217T130601.nc',
 'catalog.html?dataset=ooigoldcopy/public/CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0003_CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered_20150507T173501-20151023T193501.nc',
 'catalog.html?dataset=ooigoldcopy/public/CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0005_CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered_20160513T140200-20160808T180200.nc',
 'catalog.html?dataset=ooigoldcopy/public/CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0006_CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered_20161013T184501-20170122T060001.nc',
 'catalog.html?dataset=o

In [13]:
# Now build the url to access the data
sensor_files = [re.sub("catalog.html\?dataset=", M2M.URLS["goldCopy_dodsC"], file) for file in sensor_files]
sensor_files

['https://thredds.dataexplorer.oceanobservatories.org/thredds/dodsC/ooigoldcopy/public/CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0001_CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered_20131121T181601-20140217T130601.nc',
 'https://thredds.dataexplorer.oceanobservatories.org/thredds/dodsC/ooigoldcopy/public/CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0003_CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered_20150507T173501-20151023T193501.nc',
 'https://thredds.dataexplorer.oceanobservatories.org/thredds/dodsC/ooigoldcopy/public/CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0005_CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctdbp_cdef_instrument_recovered_20160513T140200-20160808T180200.nc',
 'https://thredds.dataexplorer.oceanobservatories.org/thredds/dodsC/ooigoldcopy/public/CP01CNSM-MFD37-03-CTDBPD000-recovered_inst-ctd

In [ ]:
# preprocess the data
zs = [preprocess(file) for file in sensor_files]

In [ ]:
# Load all the datasets
with ProgressBar():
    data = xr.concat([ds.chunk() for ds in dask.compute(*zs)], dim="time")

##### Using ooi_data_explorations modules

In [85]:
# Load data with ooi_common module
data = ooi_common.load_gc_thredds(site,node,sensor,method,stream,use_dask=True)    # Request the gold copy data through THREDDs catalog
# The potential error I see here that maybe Andrew was trying to work around was not getting ancillary files through this method
# load_gc_thredds() also calls process_file() within gc_collect() so we achieve the same preprocessing as in the preprocess() defined above.

In [86]:
# Make a copy of the data with a unique name
ds_prod = data.copy()

### QARTOD in Development: Request data from dev1 server

In [87]:
# Setup parameters needed to request data - need to set new parameters since only a subset of datasets is available from dev1 server
refdes = "CP01CNSM-MFD35-05-PCO2WB000"              # Coastal Pioneer Array (NES) - Central Surface Mooring Seafloor Multi-Function Node - pCO2 Water
method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "pco2w_abc_instrument"                     # name of data stream
# Site, node, and sensor info from deconstructed reference designator
site = "CP01CNSM"
node = "MFD35"
sensor = "05-PCO2WB000"

In [88]:
# Sub in ooinet-dev1-west.intra.oceanobservatories.org into the avaialbe API urls
Dev01_urls = {}
for key in M2M.URLS:
    url = M2M.URLS.get(key)
    if "opendap" in url:
        dev1_url = re.sub("opendap", "opendap-dev1-west.intra", url)
    else:
        dev1_url = re.sub("ooinet","ooinet-dev1-west.intra", url)
    Dev01_urls[key] = dev1_url
    
Dev01_urls
   

{'data': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12576/sensor/inv',
 'anno': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12580/anno/find',
 'vocab': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12586/vocab/inv',
 'asset': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12587',
 'deploy': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12587/events/deployment/inv',
 'preload': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12575/parameter',
 'cal': 'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12587/asset/cal',
 'fileServer': 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/fileServer/',
 'dodsC': 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/dodsC/',
 'goldCopy': 'https://thredds.dataexplorer.oceanobservatories.org/thredds/catalog/ooigoldcopy/public/',
 'goldCopy_fileServer': 'https://thredds.dataexplorer.oceanobservatories.org/thre

In [89]:
# use the Dev1 URL that is similar to the URL used in the M2M example notebook here: https://github.com/ooi-data-review/2018-data-workshops/blob/master/chemistry/examples/quickstart_python.ipynb 
api_base_url = Dev01_urls['data']
api_base_url

'https://ooinet-dev1-west.intra.oceanobservatories.org/api/m2m/12576/sensor/inv'

In [90]:

# Create the request URL
data_request_url ='/'.join((api_base_url,site,node,sensor,method,stream))

# All of the following are optional, but you should specify a date range (dropped this argument from data request below)
params = {
  'beginDT':'2018-01-01T00:00:00.000Z',
  'endDT':'2019-01-01T00:00:00.000Z',
  'format':'application/netcdf',
  'include_provenance':'true',
  'include_annotations':'true'
}

In [91]:
# Initialize credentials
import netrc
try:
    nrc = netrc.netrc()
    AUTH = nrc.authenticators('ooinet-dev1-west.intra.oceanobservatories.org')
    login, password = AUTH[0], AUTH[2]
    if AUTH is None:
        raise RuntimeError(
            'No entry found for machine ``ooinet.oceanobservatories.org`` in the .netrc file')
except FileNotFoundError as e:
    raise OSError(e, os.strerror(e.errno), os.path.expanduser('~'))

In [92]:
# Build and send the data request
r = requests.get(data_request_url, params=params, auth=(login, password))
data = r.json()
data

{'requestUUID': 'e55e5b4d-8eb5-449a-b740-ff130e31339a',
 'outputURL': 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/catalog/ooi/kylene.cooley@whoi.edu/20230410T184315707Z-CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument/catalog.html',
 'allURLs': ['https://opendap-dev1-west.intra.oceanobservatories.org/thredds/catalog/ooi/kylene.cooley@whoi.edu/20230410T184315707Z-CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument/catalog.html',
  'https://downloads-dev1-west.intra.oceanobservatories.org/async_results/kylene.cooley@whoi.edu/20230410T184315707Z-CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument'],
 'sizeCalculation': 681812,
 'timeCalculation': 60,
 'numberOfSubJobs': 14}

In [93]:
# Loading in NetCDF files

# Copy and paste one of the URLs above manually below (below I used the THREDDS catalog URL)
url = 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/catalog/ooi/kylene.cooley@whoi.edu/20230410T161638159Z-CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument/catalog.html'

# Find all available .nc files in the directory indicated by the catalog URL
tds_url = 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/dodsC' # the above URL is to the catalog/request that we built, so is this the URL that actually goes to the datasets?
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = ["/".join((tds_url, i)) for i in x] # I changed this from os.path.join because it will create paths with a "\\" delimiter instead of "/" on Windows
datasets

['https://opendap-dev1-west.intra.oceanobservatories.org/thredds/dodsC/ooi/kylene.cooley@whoi.edu/20230410T161638159Z-CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument/deployment0010_CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument_blank_20181030T020053-20190101T200052.nc',
 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/dodsC/ooi/kylene.cooley@whoi.edu/20230410T161638159Z-CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument/deployment0010_CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument_20181030T030009-20181231T230009.nc',
 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/dodsC/ooi/kylene.cooley@whoi.edu/20230410T161638159Z-CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument/deployment0009_CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument_blank_20180325T000055-20181029T060055.nc',
 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/dodsC/ooi/kylene.cooley@whoi.

In [41]:
# Remove _blank dataset files
# These are used to calculate derived parameters, but we will not be working with these in this notebook
selected_datasets = []
for d in datasets:
    if 'blank' in d:
        pass
    else:
        selected_datasets.append(d)
selected_datasets

['https://opendap-dev1-west.intra.oceanobservatories.org/thredds/dodsC/ooi/kylene.cooley@whoi.edu/20230410T161638159Z-CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument/deployment0010_CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument_20181030T030009-20181231T230009.nc',
 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/dodsC/ooi/kylene.cooley@whoi.edu/20230410T161638159Z-CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument/deployment0009_CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument_20180325T010009-20181029T120009.nc',
 'https://opendap-dev1-west.intra.oceanobservatories.org/thredds/dodsC/ooi/kylene.cooley@whoi.edu/20230410T161638159Z-CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument/deployment0008_CP01CNSM-MFD35-05-PCO2WB000-recovered_inst-pco2w_abc_instrument_20180101T000009-20180329T190009.nc']

In [55]:
# Load files into xarray dataset
ds = xr.open_mfdataset(selected_datasets)
ds = ds.swap_dims({'obs': 'time'}) # Swap the primary dimension
ds = ds.chunk({'time': 100}) # Used for optimization
ds = ds.sortby('time') # Data from different deployments can overlap so we want to sort all data by time stamp.
ds['provenance']

<xarray.DataArray 'provenance' (time: 1874)>
dask.array<rechunk-merge, shape=(1874,), dtype=|S64, chunksize=(100,), chunktype=numpy.ndarray>
Coordinates:
    obs      (time) int32 dask.array<chunksize=(100,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2018-01-01T00:00:09 ... 2018-03-29T19:00:09
Attributes:
    name:         provenance
    coordinates:  time lat lon
    _ChunkSizes:  [1874   36]

In [14]:
# # The following example is just the same as the ooinet gold copy! I'm not sure how the dev1 urls above get used.
# # We can't use any of the following because the development environment doesn't have a gold copy
# # Use the gold copy THREDDs datasets
# thredds_url = M2M.get_thredds_url(refdes, method, stream, goldCopy=True) # in this example we recycle refdes, method, stream

# # Get the THREDDs catalog
# thredds_catalog = M2M.get_thredds_catalog(thredds_url)

# # Clean the THREDDs catalog
# sensor_files, ancillary_files = M2M.clean_catalog(thredds_catalog, stream)

# # Now build the url to access the data
# sensor_files = [re.sub("catalog.html\?dataset=", M2M.URLS["goldCopy_dodsC"], file) for file in sensor_files]
# zs = [preprocess(file) for file in sensor_files]

# # Load all the datasets
# with ProgressBar():
#     data = xr.concat([ds.chunk() for ds in dask.compute(*zs)], dim="time")

In [56]:
# Preprocess with ooi_data_explorations.common.process_file() adapted for Dev01 datasets

# url = re.sub('catalog.html\?dataset=', Dev01_urls, catalog_file)
# r = SESSION.get(url, timeout=(3.05, 120))
# if r.ok:
#     if use_dask:
#         ds = xr.open_dataset(io.BytesIO(r.content), decode_cf=False, chunks=10000)
#     else:
#         ds = xr.load_dataset(io.BytesIO(r.content), decode_cf=False)
# else:
#     failed_file = catalog_file.rpartition('/')
#     warnings.warn('Failed to download %s' % failed_file[-1])
#     return None

# addresses error in how the *_qartod_executed variables are set
qartod_pattern = re.compile(r'^.+_qartod_executed$')
for v in ds.variables:
    if qartod_pattern.match(v):
        # the shape of the QARTOD executed variables should compare to the provenance variable
        if ds[v].shape != ds['provenance'].shape:
            ds = ds.drop_vars(v)

# convert the dimensions from obs to time and get rid of obs and other variables we don't need
# ds = ds.swap_dims({'obs': 'time'})
ds = ds.reset_coords()
keys = ['obs', 'id', 'provenance', 'driver_timestamp', 'ingestion_timestamp',
        'port_timestamp', 'preferred_timestamp']
for key in keys:
    if key in ds.variables:
        ds = ds.drop_vars(key)

# since the CF decoding of the time is failing, explicitly reset all instances where the units are
# seconds since 1900-01-01 to the correct CF units and convert the values to datetime64[ns] types
time_pattern = re.compile(r'^seconds since 1900-01-01.*$')
ntp_date = np.datetime64('1900-01-01')
for v in ds.variables:
    if 'units' in ds[v].attrs.keys():
        if isinstance(ds[v].attrs['units'], str):  # because some units use non-standard characters...
            if time_pattern.match(ds[v].attrs['units']):
                del(ds[v].attrs['_FillValue'])  # no fill values for time!
                ds[v].attrs['units'] = 'seconds since 1900-01-01T00:00:00.000Z'
                np_time = ntp_date + (ds[v] * 1e9).astype('timedelta64[ns]')
                ds[v] = np_time

# sort by time
ds = ds.sortby('time')

# clear-up some global attributes we will no longer be using
keys = ['DODS.strlen', 'DODS.dimName', 'DODS_EXTRA.Unlimited_Dimension', '_NCProperties', 'feature_Type']
for key in keys:
    if key in ds.attrs:
        del(ds.attrs[key])

try: 
    ds.encoding['unlimited_dims']
    del ds.encoding['unlimited_dims']
except KeyError:
    pass

# resetting cdm_data_type from Point to Station and the featureType from point to timeSeries
ds.attrs['cdm_data_type'] = 'Station'
ds.attrs['featureType'] = 'timeSeries'

# update some global attributes
ds.attrs['acknowledgement'] = 'National Science Foundation'
ds.attrs['comment'] = 'Data collected from the OOI Dev01 M2M API and reworked for use in locally stored NetCDF files.'



In [58]:
# Make a copy of the data with a unique name
ds_dev = ds.copy()
ds_dev


<xarray.Dataset>
Dimensions:                                   (time: 1874, spectrum: 14)
Coordinates:
  * spectrum                                  (spectrum) int32 0 1 2 ... 12 13
  * time                                      (time) datetime64[ns] 2018-01-0...
Data variables: (12/24)
    pco2w_thermistor_temperature_qc_executed  (time) int8 dask.array<chunksize=(100,), meta=np.ndarray>
    pco2_seawater_qartod_results              (time) int8 dask.array<chunksize=(100,), meta=np.ndarray>
    pco2_seawater_qc_executed                 (time) int8 dask.array<chunksize=(100,), meta=np.ndarray>
    absorbance_ratio_434_qc_executed          (time) int8 dask.array<chunksize=(100,), meta=np.ndarray>
    absorbance_blank_620                      (time) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    record_type                               (time) float32 dask.array<chunksize=(100,), meta=np.ndarray>
    ...                                        ...
    absorbance_ratio_620_qc_executed          (time) int8 dask.array<chunksize=(100,), meta=np.ndarray>
    record_time                               (time) datetime64[ns] dask.array<chunksize=(100,), meta=np.ndarray>
    absorbance_ratio_620_qc_results           (time) int8 dask.array<chunksize=(100,), meta=np.ndarray>
    thermistor_raw                            (time) float32 dask.array<chunksize=(100,), meta=np.ndarray>
    pco2_seawater                             (time) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    light_measurements                        (time, spectrum) float32 dask.array<chunksize=(100, 14), meta=np.ndarray>
Attributes: (12/69)
    node:                               MFD35
    comment:                            Data collected from the OOI Dev01 M2M...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             pco2w_abc_instrument
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.13338333
    lon:                                -70.7783

### Save datasets to interim data folder for further processing

In [94]:
# something along the lines of write_nc(), xr.save_dataset?
# call the data file datasets_for_testing.nc 
# I think we will test datasets in production and development separately, so maybe making separate files for those types
interim_data = os.path.relpath('../data/interim')

dev_filename = '-'.join(('dev',ds_dev.id,))+'.nc'
prod_filename = '-'.join(('prod',ds_prod.id,))+'.nc'

dev_path=os.path.join(interim_data, dev_filename)
prod_path=os.path.join(interim_data, prod_filename)

ds_dev.to_netcdf(path=dev_path)
ds_prod.to_netcdf(path=prod_path)